# Intégration du Hello WebAssembly avec npm

---

> Dans cette seconde partie, nous tâcherons de faire que notre module écrit en web assembly puisse être utilisé avec npm.

**Je pars du principe que vous connaissez déjà très bien l'écosystème Node et que vous n'aurez aucune difficulté à comprendre les appels à la commande npm.**

## La création du module npm

### Recompiler pour créer un empaqueteur

- On réutilisera la commande wasm-pack avec l'argument `bundler` a la place de `web` pour la `target` :

`wasm-pack build --target bundler`

```
.
 |-first_step_bg.wasm
 |-.gitignore
 |-first_step_bg.js
 |-first_step.js
 |-first_step_bg.wasm.d.ts
 |-first_step.d.ts
 |-package.json
```
*Normalement, par rapport au chapitre précédent. Le dossier pkg/ devrait en plus le fichier `first_step_bg.js`*

### Linker le nouveau paquet

- Nous utiliserons `npm link` pour rendre notre nouveau paquet accessible aux autres paquets javascript :

`cd pkg`

`npm link`

Nous avons maintenant un package npm, écrit en Rust, mais compilé en WebAssembly. Il est prêt à être utilisé depuis JavaScript et ne nécessite pas que l'utilisateur ait Rust installé ; le code inclus est le code WebAssembly, et non le code source Rust.

**Sur l'image Docker, la variable d'environnement NPM_CONFIG_PREFIX est fixée sur /home/jupyter/Desktop/node, c'est dans ce dossier node que vous trouverez vos paquets.**

*Donc après avoir executer `npm link`, un lien vers le paquet `first-step` devrait être présent dans le dossier `/home/jupyter/Desktop/node/lib/node_modules/`.*

```
jupyter@3681705d7b55:~/Desktop/first-step-advanced/pkg$ ls -lR /home/jupyter/Desktop/node/
/home/jupyter/Desktop/node/:
total 4
drwxr-xr-x 3 jupyter jupyter 4096 May 19 18:06 lib

/home/jupyter/Desktop/node/lib:
total 4
drwxr-xr-x 2 jupyter jupyter 4096 May 19 18:06 node_modules

/home/jupyter/Desktop/node/lib/node_modules:
total 0
lrwxrwxrwx 1 jupyter jupyter 32 May 19 18:06 first-step -> ../../../first-step/pkg
```

## Création d'une nouvelle page utilisant node

> Afin de démontrer l'indépendance entre Rust et le paquet nouvellement créer, on generera une page en dehors du dossier que nous venons d'utiliser. 

### Nouveau projet

- Créons une nouvelle page dans un dossier `first-step-site`:

`cd ${HOME}/Desktop`

`mkdir first-step-site`

`cd first-step-site`



- Une fois dans le dossier, écrivez un fichier `package.json` qui utilisera notre module `first-step` :

```
{
    "scripts": {
        "serve": "webpack-dev-server"
    },
    "dependencies": {
        "first-step": "^0.1.0"
    },
    "devDependencies": {
        "webpack": "^5.82.1",
        "webpack-cli": "^5.1.1",
        "webpack-dev-server": "^4.15.0"
    }
}
```

### Ajouter le paquet web assembly

- Linker le module first-step grâce a npm link :

`npm link first-step`

*Un lien symbolique vers le dossier `pkg/` de notre paquet sera écrit dans le sous-dossier `node_modules`.*

### Configuration

- Configurer webpack en écrivant le fichier `webpack.config.js` qui devra contenir :

```
const path = require("path");
module.exports = {
    entry: "./index.js",
    output: {
        path: path.resolve(__dirname, "dist"),
        filename: "index.js",
    },
    mode: "development",
    experiments: {
        asyncWebAssembly: true,
    },
};
```

### Le code de la page

- Puis créer un fichier `index.js` qui contiendra l'appel à notre fonction :

```
import { greet } from 'first-step';
greet("WebAssembly with npm");
```

- Enfin, fabriquez un nouveau dossier `public/` et y écrire ce fichier `index.html` :

`mkdir public`

`cd public`

Voici un exemple de contenu du fichier `index.html` :

```
<!DOCTYPE html>
<html lang="en-US">
    <head>
        <meta charset="utf-8" />
        <title>first-step example</title>
    </head>
    <body>
        <script src="./index.js"></script>
    </body>
</html>
```



## Installation et exécution

> Il ne reste plus qu'a installer tous les paquets et de lancer le serveur.

### Installation

`npm install`

```
up to date, audited 311 packages in 1s

39 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities

```

### Le script serve

`npm run serve`

```
jupyter@f8fda7c4746e:~/Desktop/first-step-site$ npm run serve

> serve
> webpack-dev-server

<i> [webpack-dev-server] Project is running at:
<i> [webpack-dev-server] Loopback: http://localhost:8081/
<i> [webpack-dev-server] Content not from webpack is served from '/home/jupyter/Desktop/first-step-site/public' directory
asset index.js 273 KiB [emitted] (name: main)
asset e6a6b79ae793cdaacf1b.module.wasm 17 KiB [emitted] [immutable]
asset first-step_pkg_first_step_js.index.js 6.38 KiB [emitted]
runtime modules 33.2 KiB 17 modules
modules by path ./ 177 KiB
  modules by path ./node_modules/webpack-dev-server/client/ 71.8 KiB 16 modules
  modules by path ./node_modules/webpack/hot/*.js 5.3 KiB 4 modules
  modules by path ./node_modules/html-entities/lib/*.js 81.3 KiB
    ./node_modules/html-entities/lib/index.js 7.74 KiB [built] [code generated]
    + 3 modules
  ./index.js 107 bytes [built] [code generated]
  ./node_modules/ansi-html-community/index.js 4.16 KiB [built] [code generated]
  ./node_modules/events/events.js 14.5 KiB [built] [code generated]
modules by path ../first-step/pkg/ 2.65 KiB (javascript) 17 KiB (webassembly)
  ../first-step/pkg/first_step.js 157 bytes [built] [code generated]
  ../first-step/pkg/first_step_bg.js 2.44 KiB [built] [code generated]
  ../first-step/pkg/first_step_bg.wasm 60 bytes (javascript) 17 KiB (webassembly) [built] [code generated]
webpack 5.83.1 compiled successfully in 424 ms

```

![NPM](pictures/realisation_npm.png)

*Vous pouvez supprimer rust de l'image Docker si vous voulez constater que la page fonctionne encore :).*

**DANS LE PROCHAIN CHAPITRE, NOUS NOUS RESSERVIRONS DE CETTE PAGE POUR COMMENCER À CODER EN RUST.**